# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库Milvus

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [16]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_milvus import Milvus, BM25BuiltInFunction
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        }

vectordb = Milvus(
    embedding_function=my_emb,
    collection_name="ZXVMAXS6",
    vector_field=["dense", "sparse"],  # dense 列 + sparse 列
    connection_args=connection_args,
    consistency_level="Strong",
    builtin_function=BM25BuiltInFunction()  # 启用 BM25
)


2025-08-15 21:26:27,203 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


In [17]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

[Document(metadata={'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'total_pages': 29, 'keywords': '', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'moddate': '', 'subject': '', 'creationDate': "D:20230523214533+08'00'", 'modDate': '', 'producer': 'Apache FOP Version 2.6', 'format': 'PDF 1.4', 'author': '', 'page': 13, 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'title': '目录', 'trapped': '', 'pk': 460125151305463120}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [18]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='qwen3:8B', temperature=0.1)

# my_llm.invoke("你好")

/tmp/ipykernel_935151/2051099669.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434', model='qwen3:8B', temperature=0.1)


## 3. 构建检索问答链

prompts

In [19]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [20]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


#### 创建一个基于模板的检索链： 混合检索版本

混合检索

Milvus自带混合检索功能，
数据写入向量数据库流程需要修改

参考： https://milvus.io/docs/zh/milvus_hybrid_search_retriever.md

修改147-156行： 

In [21]:
# 如果使用Milvus的混合检索
            # vectordb = Milvus.from_documents(
            # documents=batch_docs,
            # embedding=my_emb,
            # builtin_function=BM25BuiltInFunction(),
            # vector_field=["dense", "sparse"],
            # collection_name="ZXVMAXS2",
            # drop_old=False,
            # connection_args=connection_args,
            # consistency_level="Strong",
            # )       

In [23]:
from langchain_milvus import Milvus, BM25BuiltInFunction
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS6",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
        builtin_function=BM25BuiltInFunction(),
        vector_field=["dense", "sparse"]
    )

2025-08-15 21:27:25,506 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


定义检索器，从向量数据库检索与问题相关的文档片段

In [24]:
retriever = vectordb.as_retriever()

将检索到的多个文档合并为单一字符串，作为上下文的输入

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

​​输入处理​​：  
RunnablePassthrough()直接传递用户问题到question字段。  
retriever | format_docs：检索文档并格式化为字符串，作为context。  
​​提示生成​​：prompt将context和question填充到模板中。  
​​模型调用​​：llm生成回答。  
​​输出解析​​：StrOutputParser将模型输出转为字符串。  

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT 
    | my_llm
    | StrOutputParser()
)


In [27]:
query = "ZXVMAX-S上网日志有哪些功能？"
res = rag_chain.invoke(query)
res


'<think>\n好的，用户问的是ZXVMAX-S上网日志有哪些功能。首先，我需要仔细查看提供的上下文内容，看看有没有直接提到上网日志相关的信息。\n\n首先，在产品定位与特点部分，提到了ZXVMAX-S端到端系统有用户分析、网络分析、专题分析和评估优化四大功能。还提到支持XDR查询、实时保障、多维分析、会话跟踪等功能。这里可能涉及到日志相关的功能，比如XDR查询可能和日志有关，但需要进一步确认。\n\n接下来，在产品特点部分，分层架构、可靠性设计、数据可靠性等部分，主要讲的是系统架构和可靠性，没有直接提到上网日志。然后，在7.3数据可靠性部分，可能涉及数据存储和处理，但同样没有明确提到日志功能。\n\n再仔细看用户的问题，上网日志可能指的是系统记录的用户上网行为或网络活动日志。根据上下文，ZXVMAX-S支持XDR查询，XDR（详单记录）通常包含详细的用户使用数据，可能包括上网日志。此外，多维分析和会话跟踪也可能涉及日志数据的分析。\n\n不过，上下文中并没有明确提到“上网日志”这个具体功能名称。因此，可能需要根据现有信息推断。例如，XDR查询可能用于获取用户的上网日志，实时保障可能涉及日志监控，会话跟踪可能记录用户的连接情况。但如果没有直接提到，就不能确定这些是否属于上网日志功能。\n\n根据用户提供的上下文，没有明确列出上网日志的具体功能，因此正确的做法是回答不知道，并且最后加上“谢谢你的提问！”。\n</think>\n\n根据提供的上下文，没有明确提到ZXVMAX-S系统中“上网日志”的具体功能。因此，我无法提供相关功能的详细信息。谢谢你的提问！'

#### 加入ReRanker进行精排

第一阶段检索结束后，我们需要对候选数据重新排名，以获得更好的结果。您可以根据自己的要求选择加权排名器（WeightedRanker）或重新 排名 器（RRFRanker）。

WeightedRanker/RRF：是一种多路召回融合算法，通过加权融合不同检索路径（如向量搜索、关键词搜索）的排名结果，提升召回多样性。但它仅基于排名位置计算分数，不深入分析语义相关性。

Cohere等 Rerank：是基于深度学习的交叉编码器（Cross-Encoder），直接计算查询与文档的语义匹配分数，能更精准地识别上下文关联性，尤其适合处理复杂语义或多义词问题。

所以可以进一步进行Reranker

构建RAG

In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate


my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

my_llm = Ollama(base_url='http://localhost:11434', model='qwen3:8B', temperature=0.1)


template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 初始化重排模型（显式指定设备）
rerank_model = HuggingFaceCrossEncoder(
    # model_name="/opt/workspace/models/Qwen/Qwen3-Reranker-0.6B",
    model_name="/opt/workspace/models/BAAI/bge-reranker-base",
)

# 配置重排器（调整 top_n 和批处理大小）
base_compressor = CrossEncoderReranker(
    model=rerank_model,
    top_n=5,  # 根据业务需求调整
    # batch_size=8  # 加速批量查询
    
)

# 创建压缩检索器（结合 Milvus 和 Rerank）
compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=vectordb.as_retriever(search_kwargs={"k": 15})  # 初步召回 15 条
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT 
    | my_llm
    | StrOutputParser()
)

query = "ZXVMAX-S上网日志有哪些功能？"
res = rag_chain.invoke(query)
res

'<think>\n好的，用户问的是ZXVMAX-S上网日志有哪些功能。我需要根据提供的上下文来回答。首先，我得仔细看看上下文中的相关内容。\n\n在上下文中，4.1节提到上网日志保存，支持Gbase或HDFS存储，保存时间可配置，7天到一年，并且自动清理过期日志。这部分应该包括在功能里。\n\n然后是4.2节，上网日志查询，通过web界面，支持多种查询条件，比如时间范围加公网IP、目的IP、MSISDN、IMSI、URL，还可以组合这些条件，或者加上端口，进行更精确的查询。这部分是查询功能。\n\n接下来，4.9节省级网关对接，提到用户访问日志、上下线日志、NAT日志上传到SFTP，还有文件列表上传、用户监控、过滤接口、故障处理和文件重传接口。这些可能属于上网日志相关的功能，因为涉及日志上传和管理。\n\n4.10拨测结果自动比对功能，拨测文件上传到指定目录，系统定时扫描，比对结果存入GP数据库，并且有界面展示和下载结果。这也是上网日志的一部分，因为涉及日志的比对和处理。\n\n另外，产品定位与特点里提到端到端系统支持用户分析、网络分析等，但可能属于更广泛的功能，不一定直接属于上网日志。不过用户的问题可能希望涵盖所有相关功能，所以需要确认。\n\n还有，4.11可能提到的上网日志历史查询和云化XDR查询，但上下文里可能没有详细展开，需要看是否有提到。比如在4.10之后提到的“上网日志历史查询”和“云化上网日志XDR查询”，可能属于功能的一部分。\n\n需要确保所有提到的上网日志相关功能都被涵盖，比如保存、查询、批量导入查询、日志管理、账号角色管理、资源监控、告警管理等，但根据上下文，可能只有部分属于上网日志功能。例如，日志管理可能属于更广泛的系统管理，而上网日志相关的功能集中在保存、查询、对接、比对等。\n\n总结一下，主要功能包括：日志保存、查询、省级网关对接、拨测比对、历史查询、XDR查询。可能还有批量导入查询，但上下文里提到的是“上网日志批量导入查询”在4.3节，但原文中可能没有详细说明，需要确认是否有提到。\n\n根据提供的上下文，4.3节可能提到批量导入查询，但用户提供的上下文里可能没有详细内容，所以可能需要根据现有信息回答。因此，回答应包括保存、查询、省级网关对接、拨测比对、历史查询、XDR查询，以及可能的批量导入查询。但需要确保这些内容在上下文中确实